# Assignment 5

You have to get all the votes for all the departments and provinces and districts available in this [link](https://resultadoshistorico.onpe.gob.pe/EG2021/EleccionesPresidenciales/RePres/T). General a final table with the columns of the department, province, district, political_organization, total_votes.
The code must run and do not fail. If the code fails, it will be graded as zero. Check this.

In [1]:
#Nos aseguramos de instalar selenium y el web driver
!pip install selenium
!pip install webdriver-manager

  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/dc/72/96b5afa16908f9abc7c24b70adfd3a46c9740eb728ddfeab28379e38eaf9/selenium-4.16.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/14/fb/9299cf74953f473a15accfdbe2c15218e766bae8c796f2567c83bae03e98/trio-0.24.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio-websocket~=0.9 from https://files.pythonhosted.org/packages/48/be/a9ae5f50cad5b6f85bd2574c2c923730098530096e170c1ce7452394d7aa/trio_websocket-0.11.1-py3-none-any.whl.metadata
  Obtaining dependency information for outcome from https://files.pythonhosted.org/packages/55/8b/5ab7257531a5d830fc8000c476e63c935488d74609b50f9384a643ec0a62/outcome-1.3.0.post0-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 45.8 MB/s eta 0:00:0000:010:01
  

In [2]:
# Importamos selenium
from selenium import webdriver

# Importamos el driver
from webdriver_manager.chrome import ChromeDriverManager
import re
import time

In [3]:
from selenium.webdriver.common.by import By

In [4]:
import pandas as pd

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

#Ejecutamos el webdriver, en este caso ponemos mi ruta de acceso (Carlos)

service = Service(executable_path=r'C:\Users\Fiorella\Downloads\chromedriver-win32\chromedriver-win32\chromedriver.exe')
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

NoSuchDriverException: Message: Unable to locate or obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [ ]:
# con este código ingresamos al link
url = 'https://resultadoshistorico.onpe.gob.pe/EG2021/'
driver.get(url)

In [ ]:
# ingresamos al apartado de resumen general usando el XPATH.
resumen_general = driver.find_element( By.XPATH, "/html/body/onpe-root/onpe-home-onpe/div[1]/div/div/div/div[2]/div[1]/div/div/a/div[1]/img")
resumen_general.click()

In [ ]:
#Seleccionamos el resumen presidencial
presidencial = driver.find_element(By.XPATH, "/html/body/onpe-root/onpe-layout-container/onpe-menu/div/nav/div/div/div[2]/div/div[2]/a")
presidencial.click()

# Seleccionamos el ambito usando el ID
ambito = driver.find_element(By.ID, "select_ambito")
ambito.click()

#seleccionamos la opción peru del ambito
peru_opt = driver.find_element(By.XPATH, "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div/select/option[2]")
peru_opt.click()

In [ ]:
#Creamos una lista vacía para poner la tabla final

list_tablas = []

#Creamos un loop para que se seleccionen todos los departamentos

for idx_dpto in range (2,27): 
        dpto_opt = driver.find_element(By.XPATH, f"/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select/option[{idx_dpto}]")
        dpto_name = dpto_opt.text
        dpto_opt.click()
        
        dpto = driver.find_element(By.ID, "select_departamento")
        dpto.click()

#Creamos un loop para que se seleccionen todas las provincias

        for idx_prov in range(2, 100):
            try: 
                prov_opt = driver.find_element( By.XPATH, f'/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select/option[{idx_prov}]')
                prov_name = prov_opt.text
                prov_opt.click()
                print(prov_name)

                # seleccionar el distrito
                dist = driver.find_element( By.ID, "cod_dist")
                dist.click()

            except:
                    break


            for idx in range(2, 10000000):

                try:
                    dist_opt = driver.find_element( By.XPATH, f"/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select/option[{idx}]")
                    dist_name = dist_opt.text
                    dist_opt.click()



                    # Exatraer la tabla
                    tabla_votos = driver.find_element( By.ID, "table-scroll")

                    tabla_html = tabla_votos.get_property( 'innerHTML')

                    tabla_votos_clean = pd.read_html( tabla_html )[0]


                    # Establecer la primera fila como nombres de columna
                    tabla_votos_clean.columns = tabla_votos_clean.iloc[0]

                    # Eliminar la primera fila del DataFrame
                    tabla_votos_clean = tabla_votos_clean.iloc[1:]

                    tabla_votos_clean['Departament'] = dpto_name

                    tabla_votos_clean['Province'] = prov_name

                    tabla_votos_clean['District'] = dist_name

                    list_tablas.append( tabla_votos_clean )

                except:
                    break
            
        #     print(f"/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select/option[{idx}]")



In [ ]:
lista_final = pd.concat(list_tablas).reset_index(drop =True)

In [ ]:
lista_final